In [102]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as DTree
from sklearn.naive_bayes import BernoulliNB

#### Define functions

In [123]:
def read_file() -> DataFrame:
    file_data: DataFrame = pd.read_csv('../data/house-votes-84.csv')
    print(file_data.shape)
    print(file_data.isna().sum())
    return file_data

# TODO fix this.
def add_id_column(_data: DataFrame) -> DataFrame:
    df_size = _data.iloc[:, 0:1].size
    df2: list = [a + 1 for a in range(0, df_size)]
    # TODO fix this.
    # _data['id'] = df2
    print(_data.shape)
    print(_data.columns)
    return _data

def cleanup_data(_data: DataFrame) -> DataFrame:
    _data[_data == 'y'] = 1
    _data[_data == 'n'] = 0
    print(_data.head(5))
    return _data

def split_to_x_y(df: DataFrame) -> tuple:
    id2 = df.iloc[:, -1:]
    print(id2)
    x = df.iloc[:, 1:]
    y = df.iloc[:, 0:1]
    print(x.shape)
    print(y.shape)
    return x, y

def split_data(x: DataFrame, y: DataFrame) -> tuple:
    z = train_test_split(x, y, test_size=0.2, random_state=0)
    print(z[0].shape)
    print(z[1].shape)
    print(z[2].shape)
    print(z[3].shape)
    return z

def build_model_dt(_x_train: DataFrame, _y_train: DataFrame) -> DTree:
    _model = DTree(criterion='entropy')
    _model.fit(_x_train, _y_train)
    return _model

def build_model_nb(_x_train: DataFrame, _y_train: DataFrame) -> BernoulliNB:
    bayes_clf = BernoulliNB()
    bayes_clf.fit(_x_train, _y_train)
    return bayes_clf

# TODO fix this.
def eval_model(_model: DTree, _x_test, _y_test) -> DataFrame:
    predicted = _model.predict(_x_test)

    predicted2 = np.array(predicted)
    predicted3 = pd.DataFrame(dict(a = predicted2))

    print('y_test', _y_test)
    print('predicted', predicted3)

    merged = _y_test.join(predicted3)
    # merged = pd.merge(_y_test, predicted3)
    print(merged)
    return merged

def run_all_models(_x_train, _y_train, _x_test, _y_test):
    # Decision Tree model
    dt_model = build_model_dt(_x_train, _y_train)
    dt_conf = eval_model(dt_model, _x_test, _y_test)
    print(dt_conf)

    # Naive Bayesian model
    nb_model = build_model_nb(_x_train, _y_train)
    nb_conf = eval_model(nb_model, _x_test, _y_test)
    print(nb_conf)

def split_and_run_models(_data: DataFrame) -> None:
    x_values, y_values = split_to_x_y(_data)
    x_train, x_test, y_train, y_test = split_data(x_values, y_values)
    
    # run all models
    run_all_models(x_train, y_train, x_test, y_test)

def impute_1_drop(_data: DataFrame) -> DataFrame:
    _data.dropna(inplace=True)
    print(_data.describe())
    return _data

def impute_2_new_val(_data: DataFrame) -> DataFrame:
    _data[pd.isna(_data)] = 2
    print(_data.describe())
    return _data

def impute_3_max(_data: DataFrame) -> DataFrame:
    # _data[pd.isna(_data)] = _data.mode()
    df_size = _data.iloc[:, 0:1].size
    
    for i in range(0, df_size):
        _data.iloc[i].fillna(_data.iloc[i].mode(), inplace=True)
    
    print(_data.describe())
    return _data

In [125]:
file_data_100: DataFrame = pd.read_csv('./data/house-votes-84.csv')
print(file_data_100.shape)
print(file_data_100.isna().sum())
file_data_200 = file_data_100.dropna(inplace=False)
print(file_data_200.shape)
print(file_data_200.isna().sum())


(435, 17)
Class Name                                  0
handicapped-infants                        12
water-project-cost-sharing                 48
adoption-of-the-budget-resolution          11
physician-fee-freeze                       11
el-salvador-aid                            15
religious-groups-in-schools                11
anti-satellite-test-ban                    14
aid-to-nicaraguan-contras                  15
mx-missile                                 22
immigration                                 7
synfuels-corporation-cutback               21
education-spending                         31
superfund-right-to-sue                     25
crime                                      17
duty-free-exports                          28
export-administration-act-south-africa    104
dtype: int64
(232, 17)
Class Name                                0
handicapped-infants                       0
water-project-cost-sharing                0
adoption-of-the-budget-resolution         0
physician

#### Option #1 - ignore missing values
- About 50% (203 / 435) of the records dropped due to missing values.

In [105]:
data_1: DataFrame = read_file()
data_1 = add_id_column(data_1)
data_1 = cleanup_data(data_1)

data_1 = impute_1_drop(data_1)

split_and_run_models(data_1)

#### Option #2 - treat missing values as values

In [107]:
data_2: DataFrame = read_file()
data_2 = add_id_column(data_2)
data_2 = cleanup_data(data_2)

data_2 = impute_2_new_val(data_2)

split_and_run_models(data_2)

#### Option #3 - impute missing values

In [122]:
data_3: DataFrame = read_file()
data_3 = add_id_column(data_3)
data_3 = cleanup_data(data_3)

# TODO fix
df_size = data_3.iloc[0:1, :].size
print('df_size', df_size)
for i in range(0, df_size):
    data_3.iloc[i][pd.isna(data_3.iloc[i])] = data_3.iloc[i].mode()
    # data_3.iloc[i].fillna(data_3.iloc[i].mode(), inplace=True)
print(data_3.describe())


# data_3 = impute_3_max(data_3)

print(data_3)

# split_and_run_models(data_3)


(435, 17)
(435, 17)
Index(['Class Name', 'handicapped-infants', 'water-project-cost-sharing',
       'adoption-of-the-budget-resolution', 'physician-fee-freeze',
       'el-salvador-aid', 'religious-groups-in-schools',
       'anti-satellite-test-ban', 'aid-to-nicaraguan-contras', 'mx-missile',
       'immigration', 'synfuels-corporation-cutback', 'education-spending',
       'superfund-right-to-sue', 'crime', 'duty-free-exports',
       'export-administration-act-south-africa'],
      dtype='object')
   Class Name handicapped-infants water-project-cost-sharing  \
0  republican                   0                          1   
1  republican                   0                          1   
2    democrat                 NaN                          1   
3    democrat                   0                          1   
4    democrat                   1                          1   

  adoption-of-the-budget-resolution physician-fee-freeze el-salvador-aid  \
0                               